<a href="https://colab.research.google.com/github/DmitryAlexandrovv/infopoisk/blob/master/task5/task5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=735a0a6afbb4fc0b9a0629775504fbf471b57f7c50185521f3733aa6843e38d1
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [50]:
import operator
import os

import pymorphy2
from numpy import dot
from numpy.dual import norm
import glob
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from google.colab import drive
drive.mount('/content/drive')

def calc_cos_similarity(query_tf_idf_vector, document_tf_idf_vector):
    query_vector = list(query_tf_idf_vector.values())
    movie_vector = list(document_tf_idf_vector.values())

    if norm(query_vector) == 0 or norm(movie_vector) == 0:
        return 0

    return dot(query_vector, movie_vector) / (norm(query_vector) * norm(movie_vector))

def search(query):
    query_words = []
    for word in query.split():
        query_words.append(morph_analyzer.parse(word)[0].normal_form)
    unique_query_words= set(query_words)

    query_tf_vector = {}

    for term in unique_query_words:
        word_count_in_query = len([x for x in query_words if x == term])
        query_tf_vector.update({term: {'tf': word_count_in_query / len(query_words)}})

    result = {}

    for i, lines in lemmas_tf_idf_vector.items():
        document_tf_idf_vector = {}
        query_tf_idf_vector = {}
        query_terms = []

        for line in lines:
            line_data = line.split()
            term = line_data[0]
            idf = line_data[1]
            tf_idf = line_data[2]

            if term in unique_query_words:
                query_terms.append(term)
                query_tf_idf_vector[term] = query_tf_vector.get(term).get('tf') * float(idf)
                document_tf_idf_vector[term] = float(tf_idf)

        for term in unique_query_words:
            if term not in query_terms:
                query_tf_idf_vector[term] = 1
                document_tf_idf_vector[term] = 0

        cos_sim = calc_cos_similarity(query_tf_idf_vector, document_tf_idf_vector)
        if cos_sim != 0:
            result[i] = cos_sim

    sorted_result = sorted(result.items(), key=operator.itemgetter(1), reverse=True)
    result = dict(sorted_result)
    return result

morph_analyzer = pymorphy2.MorphAnalyzer()
lemmas_tf_idf_vector = {}

k = 100
for i in range(0, k):
    with open('drive/MyDrive/dz/lemmas_tf_idf/page-' + str(i) + '.txt') as file:
        lemmas_tf_idf_vector[i] = file.readlines()

print(search('Лайфхаки и рыба'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
{3: 0.9526201351759274, 11: 0.9455739500126588, 12: 0.9455739500126588, 21: 0.9455739500126588, 7: 0.9455739500126586, 9: 0.9455739500126586, 15: 0.9455739500126586, 18: 0.9455739500126586, 5: 0.9455739500126585, 8: 0.9455739500126585, 10: 0.9455739500126585, 13: 0.9455739500126585, 14: 0.9455739500126585, 20: 0.9455739500126585, 4: 0.9455739500126584, 6: 0.9455739500126584, 16: 0.9455739500126584, 17: 0.9455739500126584, 19: 0.9455739500126584, 27: 0.9411110156279419, 30: 0.9411110156279419, 31: 0.9411110156279419, 35: 0.9411110156279419, 36: 0.9411110156279419, 45: 0.9411110156279419, 57: 0.9411110156279419, 63: 0.9411110156279419, 22: 0.9411110156279417, 23: 0.9411110156279417, 24: 0.9411110156279417, 25: 0.9411110156279417, 26: 0.9411110156279417, 29: 0.9411110156279417, 32: 0.9411110156279417, 38: 0.9411110156279417, 39: 0.9411110156279417, 49: 0.9411110